In [1]:
# In case you do not have following packages installed, uncomment instalisation.

import pandas as pd
import numpy as np
import os
import glob
import functools
from pathlib import Path
import matplotlib.pyplot as plt

#!pip install dask[complete];
# you need to run these in case dask gives you error, it might need update.
#!pip install --upgrade pandas "dask[complete]"
#python -m pip install "dask[dataframe]" --upgrade
#import dask.dataframe as dd

from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, accuracy_score
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.inspection import permutation_importance       

from sklearn.feature_selection import SelectFromModel
from sklearn.utils import class_weight

#!pip3 install xgboost
#from xgboost import XGBClassifier

#!pip install conorm
#import conorm # for tmm normalisation

#!pip3 install pydeseq2 or pip install pydeseq2
#from pydeseq2.dds import DeseqDataSet
#from pydeseq2.ds import DeseqStats
#from pydeseq2.utils import load_example_data



#to install R :
#conda install -c r r-irkernel

#to install a library from R
#!pip install library edgeR
# pip install rpy2

### Data for Imaging of Parkinson ###

In [4]:
path3 = Path("/home/znazari/data/open_proteomic/")
path2 = Path("/home/znazari/data/")

In [6]:
%%capture
pathway = pd.read_csv(path2/"Reactome_official_11dic2023_hsa.txt",delimiter='\t',error_bad_lines=True, header=None, low_memory=False)
pathway = pathway.set_index(pathway.columns[0])
pathway

In [7]:
%%capture
# Count non-NaN values in each row
row_counts = pathway.count(axis=1)

# Add a new column 'non_nan_count' to the DataFrame
pathway['non_nan_count'] = row_counts

# Sort the DataFrame based on the 'non_nan_count' column in descending order
sorted_pathway = pathway.sort_values(by='non_nan_count', ascending=False)

# Drop the temporary 'non_nan_count' column
sorted_pathway = sorted_pathway.drop(columns='non_nan_count')

# Display the sorted DataFrame
sorted_pathway

In [8]:

# Assuming 'sorted_pathway' is your DataFrame

# Count non-NaN values in each row
row_counts = sorted_pathway.count(axis=1)

# Add a new column 'non_nan_count' to the DataFrame
sorted_pathway['non_nan_count'] = row_counts

# Exclude rows with less than 10 non-NaN values
filtered_pathway = sorted_pathway[sorted_pathway['non_nan_count'] >= 10]

# Drop the temporary 'non_nan_count' column
filtered_pathway = filtered_pathway.drop(columns='non_nan_count')

# Display the filtered DataFrame
filtered_pathway


,1,2,3,4,5,6,7,8,9,10,...,2598,2599,2600,2601,2602,2603,2604,2605,2606,2607
0,,,,,,,,,,,,,,,,,,,,,
R-HSA-162582___Signal Transduction,AAAS,AAMP,AATF,ABCA1,ABCD3,ABCG1,ABCG5,ABCG8,ABHD12,ABHD17A,...,ZDHHC9,ZFYVE16,ZFYVE9,ZNF217,ZNF512B,ZNRF3,ZRANB1,ZW10,ZWILCH,ZWINT
R-HSA-168256___Immune System,1C,8,9b,A1BG,AAAS,AAMP,ABCA13,ABCE1,ABI1,ABI2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R-HSA-1430728___Metabolism,18S rRNA,28S rRNA,5.8S rRNA,5S rRNA,AAAS,AACS,AADAC,AADAT,AANAT,AASDHPPT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R-HSA-1643685___Disease,18S rRNA,1a,28S rRNA,3a,3b,5.8S rRNA,5S rRNA,6,7SL RNA (ENSG00000222619),7SL RNA (ENSG00000222639),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R-HSA-392499___Metabolism of proteins,18S rRNA,28S rRNA,5.8S rRNA,5S rRNA,7SL RNA (ENSG00000222619),7SL RNA (ENSG00000222639),A4GNT,AAAS,AARS1,AARS2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R-HSA-9840309___Glycosphingolipid biosynthesis,B3GALNT1,B4GALNT1,CERK,CPTP,ESYT1,ESYT2,ESYT3,GLTP,UGCG,UGT8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R-HSA-1250342___PI3K events in ERBB4 signaling,BTC,ERBB4,EREG,HBEGF,NRG1,NRG2,NRG3,NRG4,PIK3CA,PIK3R1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R-HSA-203615___eNOS activation,AKT1,CALM1,CAV1,CYGB,DDAH1,HSP90AA1,LYPLA1,NOS3,SPR,ZDHHC21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Value to check for existence
value_to_check = 'rRNA'

# Check if the value exists in the specified column
if value_to_check in gin["Gene name"].values:
    print(f'The value {value_to_check} exists in Column2.')
else:
    print(f'The value {value_to_check} does not exist in Column2.')

The value rRNA does not exist in Column2.


In [9]:
ensemble = pd.read_csv(path3/'Ensemble_GeneID_TranscriptID_Symbol_mart_export.txt',delimiter='\t')
gin = ensemble[['Gene name','Gene stable ID']]
#droping the NaN value rows:
gin2 = gin.dropna(subset=['Gene name'])
gin2

,Gene name,Gene stable ID
0,MT-TF,ENSG00000210049
1,MT-RNR1,ENSG00000211459
2,MT-TV,ENSG00000210077
3,MT-RNR2,ENSG00000210082
4,MT-TL1,ENSG00000209082
...,...,...
274608,SPATA1,ENSG00000122432
274609,SPATA1,ENSG00000122432
274610,SPATA1,ENSG00000122432
274611,SPATA1,ENSG00000122432


In [11]:
# Create a dictionary mapping gene names to gene stable IDs from 'gin2'
gene_mapping = dict(zip(gin2['Gene name'], gin2['Gene stable ID']))

# Replace gene names with gene stable IDs in 'filtered_pathway'
filtered_pathway.replace({'Gene name': gene_mapping}, inplace=True)






In [12]:
filtered_pathway

,1,2,3,4,5,6,7,8,9,10,...,2598,2599,2600,2601,2602,2603,2604,2605,2606,2607
0,,,,,,,,,,,,,,,,,,,,,
R-HSA-162582___Signal Transduction,AAAS,AAMP,AATF,ABCA1,ABCD3,ABCG1,ABCG5,ABCG8,ABHD12,ABHD17A,...,ZDHHC9,ZFYVE16,ZFYVE9,ZNF217,ZNF512B,ZNRF3,ZRANB1,ZW10,ZWILCH,ZWINT
R-HSA-168256___Immune System,1C,8,9b,A1BG,AAAS,AAMP,ABCA13,ABCE1,ABI1,ABI2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R-HSA-1430728___Metabolism,18S rRNA,28S rRNA,5.8S rRNA,5S rRNA,AAAS,AACS,AADAC,AADAT,AANAT,AASDHPPT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R-HSA-1643685___Disease,18S rRNA,1a,28S rRNA,3a,3b,5.8S rRNA,5S rRNA,6,7SL RNA (ENSG00000222619),7SL RNA (ENSG00000222639),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R-HSA-392499___Metabolism of proteins,18S rRNA,28S rRNA,5.8S rRNA,5S rRNA,7SL RNA (ENSG00000222619),7SL RNA (ENSG00000222639),A4GNT,AAAS,AARS1,AARS2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R-HSA-9840309___Glycosphingolipid biosynthesis,B3GALNT1,B4GALNT1,CERK,CPTP,ESYT1,ESYT2,ESYT3,GLTP,UGCG,UGT8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R-HSA-1250342___PI3K events in ERBB4 signaling,BTC,ERBB4,EREG,HBEGF,NRG1,NRG2,NRG3,NRG4,PIK3CA,PIK3R1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R-HSA-203615___eNOS activation,AKT1,CALM1,CAV1,CYGB,DDAH1,HSP90AA1,LYPLA1,NOS3,SPR,ZDHHC21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Loading the already preprocessed data:

path2 = Path("/home/znazari/data") # where the output data will be saved at the end.

# with all the filtered genes:

ir3_rna_step_vst =  pd.read_csv(path2/'mydata_Log_CPM_filtered_bact_sex_effect_removed_RIN_covariate.txt',delimiter='\t' )

diagnosis = pd.read_csv(path2/'patients_HC_PK_diagnosis.csv')

# mapping diagnosis to zero and one.
diagnosis['COHORT_DEFINITION'] = diagnosis['COHORT_DEFINITION'].map({'Healthy Control': 0, "Parkinson's Disease": 1})

# X: feature matrix, y: the target variable

X=ir3_rna_step_vst.T
y=diagnosis['COHORT_DEFINITION']

In [9]:
ir3_rna_step_vst

,3000,3001,3002,3003,3004,3006,3007,3008,3009,3010,...,4121,4122,4123,4124,4125,4126,4135,4136,4139,41410
ENSG00000000419,3.845119,5.213262,4.837730,5.101270,4.974751,4.825708,5.011214,5.229956,5.174531,5.284571,...,4.451699,5.408345,5.247058,4.584573,4.398598,5.449524,5.015613,3.987076,5.496642,4.963686
ENSG00000000457,5.648852,6.202708,5.624314,6.067816,6.092646,6.223112,6.003440,6.010872,6.272480,6.285266,...,5.863215,6.422896,6.368654,6.114092,5.798945,6.383039,6.290690,5.179048,6.293745,6.058590
ENSG00000000460,3.817007,4.095830,4.012751,4.262379,3.969580,4.374808,4.417553,4.331596,4.675491,4.348990,...,3.957820,4.518421,4.424952,4.145217,4.194468,4.628795,4.747158,3.569694,4.025577,4.268701
ENSG00000000938,8.463275,8.810743,8.368607,8.466923,8.927912,9.043402,8.681277,8.235545,8.067889,8.631065,...,8.416943,9.104202,8.838484,9.193292,8.217039,8.772340,8.497247,8.612951,8.695999,8.676493
ENSG00000000971,1.944928,3.384911,2.522713,4.085667,3.693875,2.544924,3.968554,4.085922,4.811027,4.093739,...,2.209538,2.390307,3.974330,2.196031,2.196303,3.910535,4.403068,2.359305,2.445675,3.267041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000285219,2.721748,1.638440,3.771521,1.812421,2.549122,2.058582,2.125234,1.765596,2.069582,1.764143,...,2.495753,1.830501,1.218611,2.165035,3.126425,1.943526,0.648852,2.959793,2.179546,2.136740
ENSG00000285230,2.322872,2.517009,2.599599,2.386956,2.851771,2.737816,2.894924,2.388526,2.458358,2.695723,...,2.169890,2.481177,2.779666,2.643108,2.415556,2.438769,2.464313,2.474065,2.487454,2.544368
ENSG00000285280,4.541140,1.709681,2.196385,2.727775,2.760335,2.204436,2.655007,2.044956,1.865374,1.846651,...,2.344181,2.573584,2.253876,2.459218,3.058924,2.916741,2.501802,2.977444,2.196541,2.727432
ENSG00000285399,5.099339,5.353763,4.708896,5.048748,5.349433,5.567314,5.192088,5.066864,5.211345,5.326240,...,5.208116,5.197208,5.421681,5.656614,5.370812,5.384910,5.832674,4.774830,4.800063,5.287198


In [50]:
filtered_df = ir3_rna_step_vst[ir3_rna_step_vst.index.isin(unique_ensg['Gene stable ID'])]

In [51]:
filtered_df

,3000,3001,3002,3003,3004,3006,3007,3008,3009,3010,...,4121,4122,4123,4124,4125,4126,4135,4136,4139,41410
ENSG00000000938,8.463275,8.810743,8.368607,8.466923,8.927912,9.043402,8.681277,8.235545,8.067889,8.631065,...,8.416943,9.104202,8.838484,9.193292,8.217039,8.772340,8.497247,8.612951,8.695999,8.676493
ENSG00000000971,1.944928,3.384911,2.522713,4.085667,3.693875,2.544924,3.968554,4.085922,4.811027,4.093739,...,2.209538,2.390307,3.974330,2.196031,2.196303,3.910535,4.403068,2.359305,2.445675,3.267041
ENSG00000002330,3.571424,3.959793,3.388754,4.085483,3.756835,3.986339,3.665170,3.716128,4.060661,4.147625,...,3.368043,4.429167,3.945994,4.056662,2.965991,4.581369,4.673969,3.146942,4.272373,3.885313
ENSG00000002549,5.872790,5.908892,6.004722,6.073372,5.541370,7.682882,5.899073,5.657070,6.353882,5.949753,...,6.898440,6.502385,6.639824,5.425128,5.561972,6.668685,7.166978,6.111543,6.510586,6.325302
ENSG00000002587,0.139701,1.312890,1.736479,0.976318,1.218935,0.709119,1.053174,1.422313,2.073881,1.777967,...,1.751712,0.780928,1.562588,0.983353,1.199543,0.721738,-0.725075,1.494510,1.673137,0.951384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000271303,4.141592,5.154501,3.158052,5.093705,3.283577,5.812244,4.497693,4.390843,3.943871,4.750242,...,2.731588,4.644567,4.741196,4.482349,3.986768,5.829626,5.499389,3.893179,5.252354,4.670282
ENSG00000271605,3.253041,3.349505,4.194189,4.647757,3.770106,4.032242,3.725165,4.014289,3.882068,3.648119,...,4.301260,3.829374,4.916163,4.849648,3.918818,3.755242,4.052355,3.537703,4.062805,3.979138
ENSG00000272325,6.689842,6.335426,6.208001,6.495175,6.384606,6.810942,6.646111,6.402878,6.308627,6.737337,...,6.560920,6.703202,6.730188,6.621639,6.583015,6.837237,6.500973,6.102476,6.663363,6.564332
ENSG00000272886,5.031533,5.461262,4.897880,5.414562,5.042777,5.492468,5.444711,5.371691,5.520976,5.315246,...,5.277152,5.400093,5.523296,5.426866,5.269029,5.388664,5.499137,4.822674,5.094705,5.293258


In [52]:
X=filtered_df.T
y=diagnosis['COHORT_DEFINITION']

In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40)

#  Initialize and train the XGBoost model
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

#  Make predictions on the testing set
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

# Calculate AU-ROC score
au_roc_score = roc_auc_score(y_test, y_pred_proba)

# Print the AU-ROC score
print(f'AU-ROC Score: {au_roc_score}')

AU-ROC Score: 0.565281461833186


In [3]:
# Sample DataFrames
df1 = pd.DataFrame({'Column1': [1, 2, 3, 4, 5], 'Data1': ['A', 'B', 'C', 'D', 'E']})

df2 = pd.DataFrame({'Column2': [3, 4, 5, 6, 7], 'Data2': ['X', 'Y', 'Z', 'W', 'K']})

# Using merge to find the intersection
intersection = pd.merge(df1, df2, left_on='Column1', right_on='Column2', how='inner')

print(intersection)


   Column1 Data1  Column2 Data2
0        3     C        3     X
1        4     D        4     Y
2        5     E        5     Z


# compare between merge and isin:
in the merge we actually want to have the data from the both pandas dataframe files but when we use isin
we basiccaclly want from one dataframe those that are in common with the other one.

In [4]:
df1

,Column1,Data1
0,1,A
1,2,B
2,3,C
3,4,D
4,5,E


In [5]:
df2

,Column2,Data2
0,3,X
1,4,Y
2,5,Z
3,6,W
4,7,K


In [6]:
common_values2 = df1[df1['Column1'].isin(df2['Column2'])]


In [7]:
common_values2

,Column1,Data1
2,3,C
3,4,D
4,5,E


In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
pd.reset_option('display.max_rows')